In [8]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

In [9]:


# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [ ]:


# Define transformations for the datasets
transform = transforms.Compose([
    transforms.Resize((300, 30)),  # Resize images to 300*30
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(            # Normalize with ImageNet mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load datasets
train_dataset = datasets.ImageFolder(root="D:/AI/Weld_Aesthetical_Defects/train", transform=transform)
val_dataset = datasets.ImageFolder(root="D:/AI/Weld_Aesthetical_Defects/val", transform=transform)
test_dataset = datasets.ImageFolder(root="D:/AI/Weld_Aesthetical_Defects/test", transform=transform)


In [ ]:

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet18 model
import torchvision.models as models
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match the number of classes in your dataset
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop with tqdm progress bar
        train_loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training",colour='#00ff00')
        for images, labels in train_loop:
            images, labels = images.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Calculate statistics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Update progress bar
            train_loop.set_postfix(loss=loss.item(), accuracy=(100. * correct / total))

        # Print training statistics
        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

        # Validation loop with tqdm progress bar
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        val_loop = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation",colour='#ffff00')
        with torch.no_grad():
            for images, labels in val_loop:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                # Update progress bar
                val_loop.set_postfix(loss=loss.item(), accuracy=(100. * correct / total))

        # Print validation statistics
        val_loss = val_loss / len(val_loader)
        val_acc = 100. * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    print("Training complete.")

# Train the model
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# Save the trained model
torch.save(model.state_dict(), "resnet18_weld_defects_model.pth")
print("Model saved to resnet18_weld_defects_model.pth")



Using device: cuda


C:\Users\MSI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\MSI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch [1/10] Training: 100%|██████████| 1350/1350 [01:30<00:00, 14.84it/s, accuracy=86.7, loss=0.0044]


Epoch [1/10], Train Loss: 0.2957, Train Acc: 86.73%


Epoch [1/10] Validation: 100%|██████████| 290/290 [00:10<00:00, 27.68it/s, accuracy=87.1, loss=0.376] 


Epoch [1/10], Val Loss: 0.2898, Val Acc: 87.12%


...

Epoch [10/10] Training: 100%|██████████| 1350/1350 [01:48<00:00, 12.40it/s, accuracy=91.5, loss=0.00167]


Epoch [10/10], Train Loss: 0.1922, Train Acc: 91.50%


Epoch [10/10] Validation: 100%|██████████| 290/290 [00:10<00:00, 27.62it/s, accuracy=90.6, loss=0.304]   


Epoch [10/10], Val Loss: 0.2287, Val Acc: 90.55%
Training complete.
Model saved to resnet18_weld_defects_model.pth


SyntaxError: keyword argument repeated: desc (1533182004.py, line 131)

In [13]:
import torch

# Load the entire model and map it to the CPU
model = torch.load('resnet18_weld_defects_model.pth', map_location=torch.device('cpu'))

In [17]:
# Test the model
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    # Test loop with tqdm progress bar
    test_loop = tqdm(test_loader, desc="Testing",colour='#ff0000')
    with torch.no_grad():
        for images, labels in test_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Update progress bar
            test_loop.set_postfix(loss=loss.item(), accuracy=(100. * correct / total))

    # Print test statistics
    test_loss = test_loss / len(test_loader)
    test_acc = 100. * correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

# Evaluate the model on the test set
test(model, test_loader, criterion)

NameError: name 'test_loader' is not defined